# Training RL to do Cartpole Balancing

- 이 예제에서는 RL을 사용하여 수평으로 좌우로 움직일 수 있는 카트에서 폴의 균형을 잡는 모델을 학습
- OpenAI Gym 환경을 사용하여 폴을 시뮬레이션

In [ ]:
import sys
!{sys.executable} -m pip install gym

- CartPole 환경을 생성하고 이 환경에서 작동하는 방법을 확인
- 환경에는 다음과 같은 속성이 있음
    - 작업공간은 시뮬레이션의 각 단계에서 수행할 수 있는 가능한 작업의 집합
    - 관찰 공간은 우리가 관찰할 수 있는 공간

In [ ]:
# Gym 라이브러리를 사용하여 CartPole-v1 환경을 생성하고, 그 환경의 동작 공간과 관찰 공간을 출력

import gym # Gym 라이브러리를 호출

env = gym.make("CartPole-v1") # CartPole-v1 환경을 생성

print(f"Action space: {env.action_space}") # 환경의 동작 공간(action space)을 출력
print(f"Observation space: {env.observation_space}") # 환경의 관찰 공간(observation space)을 출력

- 시뮬레이션이 어떻게 작동하는지 확인
- 루프는 `env.step`이 종료 플래그 `done`을 반환하지 않을 때까지 시뮬레이션을 실행
- 여기서는 `env.action_space.sample()`을 사용하여 동작을 임의로 선택하므로 실험이 매우 빠르게 실패할 것으로 예상 (CartPole의 속도, 위치 또는 각도가 특정 한계를 벗어나면 CartPole 환경이 종료)

In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info, _ = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

# 관찰에 4개의 숫자가 포함되어 있음
- 카트의 위치
- 카트의 속도
- 막대 (폴)의 각도
- 막대 (폴)의 회전 속도

- 'rew'는 각 단계에서 받는 보상
    - 카트폴 환경에서는 각 시뮬레이션 단계마다 1포인트씩 보상이 주어지며, 목표는 총 보상, 즉 카트폴이 넘어지지 않고 균형을 잡는 시간을 최대화하는 것

- 강화 학습 중에 우리의 목표는 **정책** $\pi$를 훈련하는 것이며, 각 상태 $s$에 대해 어떤 액션 $a$를 취해야 하는지 알려주는, 즉 본질적으로 $a = \pi(s)$가 되는 것
    - 확률적 솔루션을 원한다면, 정책을 각 액션에 대한 확률 집합을 반환하는 것으로 생각할 수 있음 
    - $\pi(a|s)$는 상태 $s$에서 액션 $a$를 취해야 할 확률을 의미

## Policy Gradient Method

**정책 그라데이션**이라는 가장 간단한 RL 알고리즘에서는 신경망을 훈련시켜 다음 행동을 예측

In [ ]:
# PyTorch를 사용하여 간단한 신경망 모델을 정의하는 코드

import numpy as np
import torch
import gym

# 환경 설정
# env = gym.make("CartPole-v1")
env = gym.make("CartPole-v1", render_mode="human")


# 모델 설정
num_inputs = 4  # 입력 노드의 수를 4로 설정
num_actions = 2  # 출력 노드의 수를 2로 설정

model = torch.nn.Sequential(  # 신경망을 순차적으로 정의
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),  # 입력 크기 4, 출력 크기 128의 선형 레이어
    torch.nn.ReLU(),  # ReLU 활성화 함수를 적용
    torch.nn.Linear(128, num_actions, bias=False, dtype=torch.float32),  # 입력 크기 128, 출력 크기 2의 선형 레이어
    torch.nn.Softmax(dim=1)  # 출력을 확률 값으로 변환하는 소프트맥스 함수를 적용
)

- 많은 실험을 실행하고 각 실행 후 네트워크를 업데이트하여 네트워크를 학습
- 실험을 실행하고 모든 상태, 액션(및 권장 확률), 보상 등의 결과(**트레이스**)를 반환하는 함수를 정의

In [ ]:
# 에피소드를 실행하여 상태, 행동, 확률 및 보상을 수집하는 함수를 정의
# 에피소드 동안 에이전트는 환경에서 주어진 최대 단계 수만큼 행동을 취함
# 각 단계에서 모델의 출력을 기반으로 행동을 선택하고, 선택된 행동을 통해 환경을 진행

def run_episode(max_steps_per_episode=10000, render=False):
    states, actions, probs, rewards = [], [], [], []  # 각 에피소드에서 수집할 데이터를 저장할 리스트 초기화
    state = env.reset()  # 환경을 초기 상태로 재설정
    if isinstance(state, tuple):  # state가 튜플인 경우, 첫 번째 요소를 사용
        state = state[0]
    state = np.array(state, dtype=np.float32)  # ensure state is a numpy array
    for _ in range(max_steps_per_episode):
        if render:
            env.render()  # render가 True인 경우, 환경을 렌더링
        state_input = torch.from_numpy(np.expand_dims(state, 0))  # 모델 입력 준비
        action_probs = model(state_input)[0]  # 모델을 사용하여 행동 확률을 계산
        action = np.random.choice(num_actions, p=action_probs.detach().numpy())  # 행동을 확률적으로 선택
        result = env.step(action)  # 선택한 행동을 환경에 적용
        if len(result) == 5:  # 다섯 번째 값을 반환하는 경우 처리
            nstate, reward, done, _, info = result
        else:
            nstate, reward, done, info = result
        states.append(state)  # 현재 상태를 저장
        actions.append(action)  # 선택된 행동을 저장
        probs.append(action_probs.detach().numpy())  # 행동 확률을 저장
        rewards.append(reward)  # 보상을 저장
        if done:
            break  # 에피소드가 종료되면 반복문을 탈출
        if isinstance(nstate, tuple):  # nstate가 튜플인 경우, 첫 번째 요소를 사용
            nstate = nstate[0]
        state = np.array(nstate, dtype=np.float32)  # 다음 상태를 현재 상태로 업데이트
    return np.array(states), np.array(actions), np.array(probs), np.array(rewards)  # 수집된 데이터를 배열로 반환

훈련되지 않은 네트워크에서 하나의 에피소드를 실행하면 총 보상(일명 에피소드 길이)이 매우 낮다는 것을 관찰할 수 있음

In [ ]:
# run_episode 함수를 실행하여 상태(s), 행동(a), 확률(p), 보상(r) 데이터를 수집하고, 총 보상(r의 합계)을 출력
# 강화 학습 에피소드의 결과를 바탕으로 총 보상을 계산하고 출력

%pip install pygame
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}") # np.sum(r)를 사용하여 보상 배열(r)의 모든 요소를 합산하여 총 보상을 계산

- policy gradation 알고리즘의 까다로운 측면 중 하나는 **할인된 (discounted) 보상**을 사용하는 것
- 게임의 각 단계에서 총 보상 벡터를 계산하고, 이 과정에서 일부 계수 $감마$를 사용하여 초기 보상을 할인하는 것이 아이디어의 핵심
- 결과 벡터를 정규화하는데, 이는 훈련에 영향을 주는 가중치로 사용

In [ ]:
# 보상을 할인된 형태로 계산하여 반환하는 함수
# discounted_rewards 함수는 강화 학습에서 사용되며, 보상의 미래 가치를 고려하여 계산
# 옵션으로 결과를 정규화할 수 있음

eps = 0.0001 # 정규화할 때 분모가 0이 되는 것을 방지하기 위한 작은 값

# 매개변수:
# rewards: 에피소드 동안 받은 보상의 리스트
# gamma: 할인율로, 미래 보상의 가치를 현재 가치로 할인하는 데 사용 - 기본값은 0.99
# normalize: 계산된 할인된 보상을 정규화할지 여부를 지정 - 기본값은 True
def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = [] # 리스트를 역순으로 순회하면서 할인된 보상을 계산
    s = 0 # 현재까지의 할인된 보상의 합계를 저장
    for r in rewards[::-1]:
        s = r + gamma * s # 현재 보상 r에 gamma 비율로 이전의 할인된 보상을 더한 값
        ret.insert(0, s) # 계산된 할인된 보상을 리스트 ret의 맨 앞에 추가
    if normalize: # normalize가 True이면, ret 리스트의 평균을 빼고 표준편차로 나누어 정규화
        ret = (ret-np.mean(ret))/(np.std(ret)+eps) # np.std(ret) + eps를 사용하여 표준편차가 0이 되는 것을 방지
    return ret

- 300개의 에피소드를 실행하고 각 에피소드에서 다음을 수행
    - 실험을 실행하고 트레이스를 수집
    - 수행한 작업의 차이(`그래디언트`)와 예상 확률을 계산 - 차이가 적을수록 올바른 조치를 취했다고 확신할 수 있음
    - 할인된 보상을 계산하고 기울기에 할인된 보상을 곱하여 보상이 높은 단계가 낮은 단계보다 최종 결과에 더 많은 영향을 미치도록 함
    - 신경망의 예상 목표 동작은 실행 중 예측된 확률에서 일부는 가져오고 일부는 계산된 그라디언트에서 가져옮 - '알파' 매개변수를 사용하여 기울기와 보상이 어느 정도 고려되는지 결정하는데, 이를 강화 알고리즘의 *학습률*이라고 함
    - 마지막으로, 상태와 예상 행동에 대해 네트워크를 훈련하고 이 과정을 반복

In [ ]:
# PyTorch를 사용하여 모델을 학습시키기 위한 함수 train_on_batch를 정의
# Adam 최적화 알고리즘을 사용하여 모델의 매개변수를 업데이트

# Adam 옵티마이저를 사용하여 모델의 매개변수를 학습
# 학습률(lr)은 0.01로 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 배치 단위로 모델 학습
def train_on_batch(x, y):
    x = torch.from_numpy(x) # 입력 데이터를 numpy 배열에서 PyTorch 텐서로 변환
    y = torch.from_numpy(y) # 레이블 데이터를 numpy 배열에서 PyTorch 텐서로 변환
    optimizer.zero_grad() # 이전 단계의 기울기를 초기화
    predictions = model(x) # 모델을 사용하여 예측값 계산
    loss = -torch.mean(torch.log(predictions) * y) # 손실 함수 계산 (교차 엔트로피 손실 사용)
    loss.backward() # 손실 함수의 기울기를 계산 (역전파)
    optimizer.step() # 모델의 매개변수를 업데이트
    return loss # 손실 값을 반환

In [ ]:
# 300개의 에포크 동안 에피소드를 실행하여 정책 그래디언트 방법을 사용하여 강화 학습 모델을 학습
# 학습 과정에서 상태, 행동, 확률 및 보상을 수집하고, 각 에포크마다 모델을 업데이트

alpha = 1e-4 # 학습률 설정

history = [] # 각 에포크의 총 보상을 저장할 리스트
for epoch in range(300):
    states, actions, probs, rewards = run_episode() # run_episode()를 통해 상태, 행동, 확률, 보상을 수집
    one_hot_actions = np.eye(2)[actions]  # actions를 원-핫 인코딩하여 one_hot_actions 생성
    gradients = one_hot_actions - probs
    dr = discounted_rewards(rewards)
    dr = np.expand_dims(dr, axis=1)  # dr의 형상을 (27,)에서 (27, 1)로 변환
    gradients *= dr # dr의 형상을 (27, 1)로 변환하여 gradients와 곱함
    target = alpha * gradients + probs  # target 계산: alpha * gradients + probs
    train_on_batch(states, target) # 모델 학습
    history.append(np.sum(rewards)) # 각 에포크의 총 보상을 history에 저장
    if epoch % 100 == 0: # 100 에포크마다 현재 에포크와 총 보상을 출력
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history) # history 리스트를 이용하여 학습 과정에서의 총 보상의 변화를 시각화
plt.xlabel('Epoch')
plt.ylabel('Total Reward')
plt.show()

Now let's run the episode with rendering to see the result:

In [ ]:
_ = run_episode(render=True)